# GLM data prep

Create a table of data for each recording.
Each row is a millisecond (data only from bouts).
Variables include speeds + shifted speeds, curvature of the track, firing rate...


In [1]:
# imports
import sys
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import interpolate
from fcutils.progress import track
from loguru import logger
from scipy import stats
from fcutils.maths.signals import rolling_mean
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

sys.path.append("./")
sys.path.append(r"C:\Users\Federico\Documents\GitHub\pysical_locomotion")


from analysis.ephys.utils import get_recording_names, get_data, get_session_bouts, trim_bouts

save_folder = Path(r"D:\Dropbox (UCL)\Rotation_vte\Locomotion\analysis\ephys")

cache = Path(r"D:\GLM\data")

REGION = "CUN/PPN"
recordings = get_recording_names(region=REGION)
recordings

Connecting root@127.0.0.1:3306


[07/15/22 12:09:31] DEBUG    Connected to database: LOCOMOTION4                ]8;id=629358;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\data\dbase\__init__.py\__init__.py]8;;\:]8;id=416321;file://C:\Users\Federico\Documents\GitHub\pysical_locomotion\data\dbase\__init__.py#11\11]8;;\

array(['FC_210715_AAA1110750_r5_hairpin',
       'FC_210716_AAA1110750_r6_hairpin', 'FC_210720_AAA1110750_hairpin',
       'FC_210721_AAA1110750_hairpin', 'FC_210722_AAA1110750_hairpin',
       'FC_211022_BAA110516_hairpin', 'FC_211027_BAA110516_hairpin',
       'FC_211214_BAA110517_hairpin', 'FC_220114_BAA110517_hairpin',
       'FC_220117_BAA110517_hairpin', 'FC_220119_BAA110517_hairpin',
       'FC_220120_BAA110517_hairpin', 'FC_210917_BAA1110279_hairpin',
       'FC_210820_BAA1110281_hairpin', 'FC_210829_BAA1110281_hairpin',
       'FC_210830_BAA1110281_hairpin', 'FC_210831_BAA1110281_hairpin',
       'FC_210901_BAA1110281_hairpin', 'FC_210906_BAA1110281_hairpin',
       'FC_210917_BAA1110281_hairpin'], dtype=object)

## Parameters

In [2]:
minimum_bout_ds = 100

### Collect data for all bouts

In [7]:
recordings = get_recording_names(region=REGION)

for REC in recordings:
    savepath = cache / f"{REC}_bouts.h5"
    if savepath.exists():
        print(f"{REC}_bouts.h5 already exists")
        continue
    
    print(f"Doing   {REC}")
    
    bouts = trim_bouts(get_session_bouts(REC, complete=None,direction=None))
    n = len(bouts)
    bouts["ds"] = [abs(b.s[-1]-b.s[0]) for i,b in bouts.iterrows()]
    bouts = bouts.loc[bouts.ds >= minimum_bout_ds]


    bouts_files = [f for f in cache.glob(f"{REC}_bout_*.feather") if "xy." not in f.name]
    
    if len(bouts_files) < len(bouts):
        print(f"    Not all bouts were saved for {REC}: {len(bouts_files)}/{len(bouts)}")
        continue
    else:
        print(f"    All bouts were saved for {REC}: {len(bouts_files)}/{len(bouts)}")

    bouts_data = []
    for i, bout in bouts.iterrows():
        try:
            f = cache / f"{REC}_bout_{bout.start_frame}.feather"
            _data = pd.read_feather(f)
        except:
            logger.warning(f"Failed to read file for bout {i}: {f}")
            continue
        bouts_data.append(_data.astype(np.float32))


    if not len(bouts_data) or len(bouts_data) < len(bouts):
        print(f"    No bouts were saved for {REC}")
        continue
    bouts_data = pd.concat(bouts_data)

    if bouts_data.shape[0] < 10000:
        print(f"    Too few frames for {REC}: {bouts_data.shape[0]}")
        continue

    print(f" Saving data (shape: {bouts_data.shape})")
    bouts_data.to_hdf(savepath, key="data")
    print(" Saved all data")


FC_210715_AAA1110750_r5_hairpin_bouts.h5 already exists
FC_210716_AAA1110750_r6_hairpin_bouts.h5 already exists
FC_210720_AAA1110750_hairpin_bouts.h5 already exists
Doing   FC_210721_AAA1110750_hairpin
    Not all bouts were saved for FC_210721_AAA1110750_hairpin: 0/60
FC_210722_AAA1110750_hairpin_bouts.h5 already exists
FC_211022_BAA110516_hairpin_bouts.h5 already exists
FC_211027_BAA110516_hairpin_bouts.h5 already exists
FC_211214_BAA110517_hairpin_bouts.h5 already exists
FC_220114_BAA110517_hairpin_bouts.h5 already exists
FC_220117_BAA110517_hairpin_bouts.h5 already exists
FC_220119_BAA110517_hairpin_bouts.h5 already exists
FC_220120_BAA110517_hairpin_bouts.h5 already exists
FC_210917_BAA1110279_hairpin_bouts.h5 already exists
Doing   FC_210820_BAA1110281_hairpin
    All bouts were saved for FC_210820_BAA1110281_hairpin: 0/0
    No bouts were saved for FC_210820_BAA1110281_hairpin
Doing   FC_210829_BAA1110281_hairpin
    All bouts were saved for FC_210829_BAA1110281_hairpin: 2/2
   